# Database Engineering

In [8]:
# Read in dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [9]:
# Create an engine to a SQLite database called 'wine.sqlite'
engine = create_engine("sqlite:///wine_reviews.sqlite")

In [10]:
# Create a connection to the engine
conn = engine.connect()

## Creating Table

In [4]:
# Use declarative_base to model the measurements 
Base = declarative_base()

class Reviews(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    country = Column(Text)
    description = Column(Text)
    points = Column(Integer)
    price = Column(Integer)
    province = Column(Text)
    region_1 = Column(Text)
    title = Column(Text)
    variety = Column(Text)
    winery = Column(Text)
    latitude = Column(Integer)
    longitude = Column(Integer)

In [5]:
# Create the Reviews table in the database
Base.metadata.create_all(engine)

In [11]:
# Load the cleaned csv file 
df = pd.read_csv("raw_data/raw_data/coordinates.csv", index_col=0, encoding='utf-8')
df.head()

,country,description,points,price,province,region_1,title,variety,winery,latitude,longitude
0,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,45.344287,-122.667037
1,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,45.344287,-122.667037
2,US,"A stiff, tannic wine, this slowly opens and br...",86,22.0,Oregon,Willamette Valley,Hawkins Cellars 2009 Pinot Noir (Willamette Va...,Pinot Noir,Hawkins Cellars,45.344287,-122.667037
3,US,This wine is bursting with gorgeous cherry fru...,91,38.0,Oregon,Willamette Valley,Le Cadeau 2014 Pinot Noir (Willamette Valley),Pinot Noir,Le Cadeau,45.344287,-122.667037
4,US,"There is an odd, piercing edge to the aromas, ...",85,28.0,Oregon,Willamette Valley,Silvan Ridge 2006 Reserve Pinot Noir (Willamet...,Pinot Noir,Silvan Ridge,45.344287,-122.667037


{'Oregon': 4928, 'Michigan': 105, 'California': 33654, 'Virginia': 725, 'Washington': 7965, 'New York': 2456, 'Idaho': 26, 'Texas': 82, 'Pennsylvania': 17, 'New Jersey': 8, 'Missouri': 33, 'New Mexico': 44, 'Nevada': 3, 'Colorado': 62, 'Arizona': 39, 'Massachusetts': 6, 'Rhode Island': 1, 'Ohio': 10, 'Illinois': 5, 'North Carolina': 22, 'Iowa': 4, 'Vermont': 2, 'Kentucky': 1, 'Connecticut': 3, 'Hawaii': 1}


In [12]:
# Change wine_df to a dictionary
# orient='records' makes the dictionary list-like [{column -> value}]
data_reviews = df.to_dict(orient='records')

In [13]:
# Data is now a list of dictionaries that represent each row of data.
data_reviews[0]

{'country': 'US',
 'description': 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'latitude': 45.3442868,
 'longitude': -122.6670374,
 'points': 87,
 'price': 14.0,
 'province': 'Oregon',
 'region_1': 'Willamette Valley',
 'title': 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 'variety': 'Pinot Gris',
 'winery': 'Rainstorm'}

In [14]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the referenct to the 'reviews' table
reviews_table = sqlalchemy.Table('reviews', metadata, autoload=True)

In [16]:
# Delete any pre-existing table
# DO NOT USE STEP IN PRODUCTION
conn.execute(reviews_table.delete())

In [17]:
# Insert data into the table
conn.execute(reviews_table.insert(), data_reviews)

In [18]:
# Test the first 5 rows
conn.execute('SELECT * FROM reviews LIMIT 5').fetchall()

[(1, 'US', 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.', 87, 14, 'Oregon', 'Willamette Valley', 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'Pinot Gris', 'Rainstorm', 45.3442868, -122.6670374),
 (2, 'US', "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.", 87, 65, 'Oregon', 'Willamette Valley', "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)", 'Pinot Noir', 'Sweet Cheeks', 45.3442868, -122.6670374),
 (3, 'US', "A stiff, tannic wine, this slowly opens and brings brambly berry flavors into play, along with notes of earthy herbs. There's a touch of bitterness to the tannins.", 86, 22, 'Oregon', 'Willamette Va